In [1]:
import os
print os.getcwd()

/abaghel/projects/rnn-patient-sentiment-paper


## Prototype the Training 

In [2]:
import cPickle
import numpy as np

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape, Merge
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta
from keras.constraints import unitnorm, maxnorm
from keras.regularizers import l2
from keras.layers.advanced_activations import PReLU
from keras.preprocessing import sequence

from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU

from sklearn.metrics import roc_auc_score

# This import to resolve some errors with tf version on office server
import tensorflow as tf
tf.python.control_flow_ops = tf

Using TensorFlow backend.


### Define data preparation functions


In [3]:
def get_idx_from_sent(sent, word_idx_map, max_l=51, kernel_size=5):
    """
    Transforms sentence into a list of indices. Pad with zeroes.
    """
    x = []
    words = sent.split()
    for word in words:
        if word in word_idx_map:
            x.append(word_idx_map[word])
    while len(x) < max_l:
        x.append(0)
    return x

def make_idx_data(revs, word_idx_map, max_l=51, kernel_size=5):
    """
    Transforms sentences into a 2-d matrix.
    """
    train, val, test = [], [], []
    for rev in revs:
        sent = get_idx_from_sent(rev['text'], word_idx_map, max_l, kernel_size)
        sent.append(rev['y'])
        if rev['split'] == 1:
            train.append(sent)
        elif rev['split'] == 0:
            val.append(sent)
        else:
            test.append(sent)
    train = np.array(train, dtype=np.int)
    val = np.array(val, dtype=np.int)
    test = np.array(test, dtype=np.int)
    return [train, val, test]

### Load data 

In [4]:
print "loading data..."
x = cPickle.load(open("bigger_health-train-val-test2.pickle", "rb"))
revs, W, word_idx_map, vocab = x[0], x[1], x[2], x[3]
print "data loaded!"


datasets = make_idx_data(revs, word_idx_map, max_l=247, kernel_size=5)


loading data...
data loaded!


In [60]:
# Train data preparatio
N = datasets[0].shape[0]
conv_input_width = W.shape[1]
conv_input_height = int(datasets[0].shape[1]-1)

In [6]:
print "N: ", N
print "conv_input_height: ", conv_input_height
print "conv_input_width: ", conv_input_width

N:  27155
conv_input_height:  247
conv_input_width:  300


### Train Data Prepapration

In [7]:
# For each word write a word index (not vector) to X tensor
train_X = np.zeros((N, conv_input_height), dtype=np.int)
train_Y = np.zeros((N, 2), dtype=np.int)
for i in xrange(N):
    for j in xrange(conv_input_height):
        train_X[i, j] = datasets[0][i, j]
    train_Y[i, datasets[0][i, -1]] = 1
    
    
print 'train_X.shape = {}'.format(train_X.shape)
print 'train_Y.shape = {}'.format(train_Y.shape)

train_X.shape = (27155, 247)
train_Y.shape = (27155, 2)


In [8]:
print train_X[0] , type(train_X) , type(train_X[0])
print train_Y[0] , type(train_Y) , type(train_Y[0])

[30359 18897 32378  8607 30057 24613 35151 17688 13397   437 28698  7844
 23124  9241 32378 13814 28736 10380 30196     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [9]:
np.sum(train_Y) == train_Y.shape[0]

True

In [10]:
(np.sum(train_Y, axis=0)/float(np.sum(train_Y)))*100

array([ 31.1765789,  68.8234211])

### Prepare validation data

In [44]:
Nv = datasets[1].shape[0]

val_X = np.zeros((Nv, conv_input_height), dtype=np.int)
val_Y = np.zeros((Nv, 2), dtype=np.int)
for i in xrange(Nv):
    for j in xrange(conv_input_height):
        val_X[i, j] = datasets[1][i, j]
    val_Y[i, datasets[1][i, -1]] = 1
    
print 'val_X.shape = {}'.format(val_X.shape)
print 'val_Y.shape = {}'.format(val_Y.shape)

val_X.shape = (6845, 247)
val_Y.shape = (6845, 2)


In [45]:
print val_X[0] , type(val_X) , type(val_X[0])
print val_Y[0] , type(val_Y) , type(val_Y[0])

[13287   431 11571   112 16982  1204  1074 28736 15865 30613  6482 13814
 11571 24777 13194 20697 30352 31539 19006 31539 29072 13033 10002 23027
 31539 31983  2943 22730 16643 20903 16859 12153 27346 20722 31539 27346
 31983  2943  6892 28022 13683 22146 13287   431 17956 24441 23777  8607
 12326 28736 27346 18238 13814 19103 19016  9375 11034 18508 35234 13287
  9995 28374 20939  9375 30359 13479 17789 15865 27346 10898 19103  5248
 31540 12153 13814 11571  9713  8429 30359  1226 29976 27346 18897 11520
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [46]:
np.sum(val_Y) == val_Y.shape[0]

True

In [47]:
(np.sum(val_Y, axis=0)/float(np.sum(val_Y)))*100

array([ 31.43900657,  68.56099343])

This value above shows the class bias. 68.56% accuracy on validation should be expected even if your output for any input is one.

## Create a Keras model

In [15]:
# These will be used in the Embedding layers as paramteres
print "input_dim=", W.shape[0]
print "output_dim=", W.shape[1]
print "weights=", [W]
print "W_constraint=", unitnorm()

input_dim= 35972
output_dim= 300
weights= [array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  7.41578788e-02,  -1.86807379e-01,   1.21926129e-01, ...,
         -1.32260442e-01,   3.41992006e-02,   2.10501035e-04],
       [  1.43194227e-02,   6.02296554e-02,   2.19316810e-01, ...,
          8.92717242e-02,   4.24648672e-02,   2.71095056e-02],
       ..., 
       [ -2.34375000e-01,   2.14843750e-01,  -1.62109375e-01, ...,
          1.59179688e-01,   3.54003906e-02,  -1.13281250e-01],
       [ -1.21112600e-01,  -1.87677279e-01,  -2.32269242e-01, ...,
         -1.98359162e-01,  -1.15148671e-01,  -8.49112794e-02],
       [ -4.46777344e-02,   6.73828125e-02,  -1.41601562e-01, ...,
         -1.65039062e-01,   8.98437500e-02,  -5.51757812e-02]], dtype=float32)]
W_constraint= <keras.constraints.UnitNorm object at 0x2d52b90>


In [15]:
model = Sequential()
model.add(Embedding(input_dim=W.shape[0], output_dim=W.shape[1], weights=[W], W_constraint=unitnorm()))

In [16]:
model.output_shape

(None, None, 300)

In [17]:
model.add(LSTM(250))
model.add(Activation('relu'))

In [18]:
model.output_shape

(None, 250)

In [19]:
model.add(Dense(2))
model.add(Activation('sigmoid'))

In [20]:
model.output_shape

(None, 2)

### Compile the model

In [21]:
#optimizer is adadelta and loss function is categorical_crossentropy
model.compile(loss='categorical_crossentropy', optimizer='adadelta',
              metrics=['accuracy'])

### Fit the model

In [22]:
model.fit(train_X, train_Y, batch_size=50, nb_epoch=10, validation_data=(val_X, val_Y))

Train on 27155 samples, validate on 6845 samples
Epoch 1/10
27155/27155 [==============================] - 1269s - loss: 0.6229 - acc: 0.6876 - val_loss: 0.6227 - val_acc: 0.6856
Epoch 2/10
27155/27155 [==============================] - 840s - loss: 0.6211 - acc: 0.6882 - val_loss: 0.6234 - val_acc: 0.6856
Epoch 3/10
27155/27155 [==============================] - 772s - loss: 0.6210 - acc: 0.6882 - val_loss: 0.6228 - val_acc: 0.6856
Epoch 4/10
27155/27155 [==============================] - 805s - loss: 0.6212 - acc: 0.6882 - val_loss: 0.6226 - val_acc: 0.6856
Epoch 5/10
27155/27155 [==============================] - 780s - loss: 0.6210 - acc: 0.6882 - val_loss: 0.6226 - val_acc: 0.6856
Epoch 6/10
27155/27155 [==============================] - 770s - loss: 0.6209 - acc: 0.6882 - val_loss: 0.6226 - val_acc: 0.6856
Epoch 7/10
27155/27155 [==============================] - 834s - loss: 0.6208 - acc: 0.6882 - val_loss: 0.6226 - val_acc: 0.6856
Epoch 8/10
27155/27155 [=======================

## Evaluate model on test set

### Test data preparation2

In [48]:
Nt = datasets[2].shape[0]

test_X = np.zeros((Nt, conv_input_height), dtype=np.int)
test_Y = np.zeros((Nt, 2), dtype=np.int)
for i in xrange(Nt):
    for j in xrange(conv_input_height):
        test_X[i, j] = datasets[2][i, j]
    test_Y[i, datasets[2][i, -1]] = 1    
        
    
print 'test_X.shape = {}'.format(test_X.shape)
print 'test_Y.shape = {}'.format(test_Y.shape)


test_X.shape = (2683, 247)
test_Y.shape = (2683, 2)


In [52]:
(np.sum(test_Y, axis=0)/float(np.sum(test_Y)))*100

array([   0.,  100.])

In [ ]:
score, acc = model.evaluate(test_X, test_Y,
                            batch_size=10)

print('Test score:', score)
print('Test accuracy:', acc)

# Try another model - imdb lstm from Keras examples

In [20]:
model = Sequential()
model.add(Embedding(input_dim=W.shape[0], output_dim=W.shape[1], weights=[W], W_constraint=unitnorm()))
model.add(LSTM(300, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model.add(Dense(2))
model.add(Activation('sigmoid'))

In [21]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
model.fit(train_X, train_Y, batch_size=50, nb_epoch=10, validation_data=(val_X, val_Y))

# =============================================================
# Model is not learning

In [36]:
train_Y

array([[0, 1],
       [0, 1],
       [0, 1],
       ..., 
       [0, 1],
       [1, 0],
       [1, 0]])

In [32]:
print type(train_Y)
print type(train_Y[0])
print type(train_Y[0][0])

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.int64'>


In [33]:
train_Y.shape

(27155, 2)

In [41]:
one_value = []
for n in range(train_Y.shape[0]):
    one_value.append(train_Y[n][1])

new_train_Y = np.array(one_value, int)

In [40]:
new_train_Y

array([1, 1, 1, ..., 1, 0, 0])

## Do the same with test and val

In [42]:
one_value = []
for n in range(val_Y.shape[0]):
    one_value.append(val_Y[n][1])

new_val_Y = np.array(one_value, int)
new_val_Y

array([0, 1, 1, ..., 1, 1, 0])

In [50]:
one_value = []
for n in range(test_Y.shape[0]):
    one_value.append(test_Y[n][1])

new_test_Y = np.array(one_value, int)
new_test_Y

array([1, 1, 1, ..., 1, 1, 1])

In [63]:
model = Sequential()
model.add(Embedding(input_dim=W.shape[0], output_dim=W.shape[1], weights=[W], W_constraint=unitnorm(), dropout=0.2))
model.add(LSTM(300, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [64]:
model.fit(train_X, new_train_Y, batch_size=50, nb_epoch=10, validation_data=(val_X, new_val_Y))

/usr/lib/python2.7/site-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 27155 samples, validate on 6845 samples
Epoch 1/10
27155/27155 [==============================] - 1069s - loss: 0.6253 - acc: 0.6869 - val_loss: 0.6228 - val_acc: 0.6856
Epoch 2/10
27155/27155 [==============================] - 1073s - loss: 0.6228 - acc: 0.6882 - val_loss: 0.6241 - val_acc: 0.6856
Epoch 3/10
 3050/27155 [==>...........................] - ETA: 875s - loss: 0.6318 - acc: 0.6751

KeyboardInterrupt: 